# Context

Banking Fraud has grown strongly in recent years. 
To solve this problem I use a machine learning based approch  to recognize fraudulent credit card transactions.

**Outline :**
1. Data collection and Exploration
2. Data Preprocessing for ML
3. Modelisation and validation 
4. Choice of the king Model
5. Model saving 

# Plugins importing 

In this section, I import all plugin necessary

In [1]:

import pandas as pd
import pickle
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# 1.Data Collection and Exploration

##  1.1 Data Collection 

In this NoteBook, I use a Data Set on credit card transactions. It is an anonymized credit card transactions labeled as fraudulent or genuine. In this Data Set we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

The dataset has been collected and analysed during a research collaboration of Worldline and the Machine Learning Group (http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection.



In [2]:
#Data collection 
df = pd.read_csv("creditcard.csv")





## 1.2 Data Exploration

In [3]:
#Dysplay collumn 
list(df.columns)

['Time',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'Amount',
 'Class']

In [6]:
#Plot five fist collumn of data set
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
#Plot data size
df.shape

(284807, 31)

In [8]:
#Verify null values for collumns
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

**Note:** I notice there is no null values in the Data Set.

The Time and Amount are not important for our model. so I will remove these from the Data Set

In [7]:
#Features extractions
df_data =  df.drop("Time", axis='columns')
df_data =  df_data.drop("Amount", axis='columns')
df_data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0


**Note:** I removed the Time and Amount collumns from the Data Set

In [8]:
#Display the repartition of data in Class
counts= df_data["Class"].value_counts()
counts

0    284315
1       492
Name: Class, dtype: int64

**Note:** I notice that the CLASS is imbalaced. I can balace the Data Set with SMOTE but It is not a Good Idea for a real project. So, I will work with a imbalaced Data Set.

## 2. Data preprocessing for Machine Learning 

In [9]:
# Extract the labal
df_y = df_data.Class
# Plot five first rows
df_y.head()
df_y.shape

(284807,)

In [10]:
y_col = 'Class'
df_x=df_data[df_data.columns.drop(y_col)]
df_x.shape

(284807, 28)

In [11]:
# Split Data into train and test
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.25, random_state = 42)
target_names = ['No Fraud', 'Fraud']

# 3. Modelisation and validation 

In this part I will use the following algorithms: Logistic Regression, Random Forest and SVM.


**Metrics** 

**Recall:** corresponds to the number of transactions correctly attributed to class i compared to the total number of transactions belonging to class i (total true positive).

**Precision:** corresponds to the number of transactions correctly attributed to class i compared to the total number of transactions  predicted as belonging to class i (total predicted positive).

**F1-SCore:**  is calculated by harmonic mean Precision and Reccall

##  3.1 Logistic Regression 

In [12]:

# Train with Logistic Regression
pipe_logistic = make_pipeline(StandardScaler(), LogisticRegression())
pipe_logistic.fit(X_train, y_train) 
# Test with Logistic Regression and compute the score
pipe_logistic.score(X_test, y_test)
y_pred_logistic =pipe_logistic.predict(X_test)
# Plot classification report
classification_report(y_test,y_pred_logistic,target_names=target_names)

'              precision    recall  f1-score   support\n\n    No Fraud       1.00      1.00      1.00     71089\n       Fraud       0.85      0.59      0.70       113\n\n    accuracy                           1.00     71202\n   macro avg       0.92      0.80      0.85     71202\nweighted avg       1.00      1.00      1.00     71202\n'

## 3.2 Random Forest

In [13]:
# Train with Random Forest
pipe_random = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=100))
pipe_random.fit(X_train, y_train) 
# Test with Random Forest and compute the score
pipe_random.score(X_test, y_test)
y_pred_random=pipe_random.predict(X_test)
# Plot classification report
classification_report(y_test,y_pred_random,target_names=target_names)

'              precision    recall  f1-score   support\n\n    No Fraud       1.00      1.00      1.00     71089\n       Fraud       0.92      0.81      0.86       113\n\n    accuracy                           1.00     71202\n   macro avg       0.96      0.90      0.93     71202\nweighted avg       1.00      1.00      1.00     71202\n'

## 3.3 SVM (Support Vector Machine)

In [16]:
# Train with SVM
pipe_svm = make_pipeline(StandardScaler(), svm.SVC())
pipe_svm.fit(X_train, y_train) 
# Test with SVM and compute the score
pipe_svm.score(X_test, y_test)
y_pred_svm= pipe_svm.predict(X_test)
# Plot classification report
classification_report(y_test,y_pred_svm,target_names=target_names)

'              precision    recall  f1-score   support\n\n    No Fraud       1.00      1.00      1.00     71089\n       Fraud       0.92      0.61      0.73       113\n\n    accuracy                           1.00     71202\n   macro avg       0.96      0.81      0.87     71202\nweighted avg       1.00      1.00      1.00     71202\n'

# 4. Choice of the King Model

My main goal is to detect effectively a Fraud Transaction. That's 1 Class value. The best model is Random Forest because it has the higher f1-score(0.96)

# 5. Model Saving

In [17]:
    with open(r"MODEL.pickle", "wb") as output_file:
        pickle.dump(pipe_random, output_file)